In [ ]:
import numpy as np
from IPython.core.debugger import set_trace
from scipy.io import loadmat
import pickle
import os
import sys
import nltk
from scipy import stats
from sklearn.model_selection import KFold
from scipy.stats import spearmanr, pearsonr
import spacy
import matplotlib.pyplot as plt
from tqdm import tqdm
from spacy import displacy
from collections import defaultdict
import torch
from transformers import *
from sklearn.decomposition import PCA

np.random.seed(97)
torch.manual_seed(97)

save_dir = "features"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [ ]:
hp_text = open("chapter9.txt","r").readlines()
hp_text = [i.strip() for i in hp_text]

sent_text = open("sentences.txt", "r").readlines()
all_sentences = [""]
for i in range(len(sent_text)):
    sent_text[i] = sent_text[i].strip()
    if len(sent_text[i]) == 0 or sent_text[i] == "+":
        all_sentences.append("")
    elif sent_text[i]!="+":
        all_sentences[-1] += sent_text[i] + " "

all_sentences_clean = [i.strip() for i in all_sentences if len(i.strip()) > 0]
all_sentences = all_sentences_clean

In [ ]:
all_contexts = []
max_len = 0

for sent in all_sentences:
    words = sent.strip().split(" ")
    max_len = max(max_len, len(words))
    for i in range(len(words)):
        if (len(all_contexts) != 0  and hp_text[len(all_contexts)] == "+"):
            all_contexts.append("")
        if (len(all_contexts) != 0  and hp_text[len(all_contexts)] == "+"):
            all_contexts.append("")
        all_contexts.append(" ".join(words[:i+1]))
            
all_contexts.append([[""]])

In [ ]:
print(len(all_contexts))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
model = BertModel.from_pretrained('bert-large-cased', output_hidden_states=True)
model.eval()

In [ ]:
all_embeddings = []
selected_layer = 11

for i in range(len(all_contexts)):
    print(hp_text[i])
    print(all_contexts[i])
    if hp_text[i] == "+":
        all_embeddings.append(np.zeros(1024))
    else:
        input_ids = torch.tensor([tokenizer.encode(all_contexts[i], add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
        all_outputs = model(input_ids)
        all_hidden_states = all_outputs[2]
        selected_hidden_states = all_hidden_states[selected_layer]
        selected_hidden_states = selected_hidden_states.reshape((selected_hidden_states.shape[1], selected_hidden_states.shape[2]))
        mean_hidden_states = selected_hidden_states.mean(axis = 0).detach().numpy()
        all_embeddings.append(mean_hidden_states)

In [ ]:
np.array(all_embeddings).shape

In [ ]:
all_embeddings = np.load(os.path.join(save_dir,"incremental_bert_embeddings_layer12.npy"))
pca = PCA(n_components=15)
np.random.seed(97)

reduced = pca.fit_transform(np.array(all_embeddings))
np.save(os.path.join(save_dir,"incremental_bert_embeddings_layer12_PCA_dims_15"), reduced)

In [ ]:
reduced.shape